In [1]:
# import required libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import QuantileRegressor
from sklearn.metrics import mean_pinball_loss
import matplotlib.pyplot as plt

In [2]:
# read the dataset
df = pd.read_csv(r"D:\flood prediction\imputated weather dataset.csv")
df.head()

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Flood?
0,Barisal,1949.0,1.0,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-01,0.0
1,Barisal,1949.0,2.0,33.9,15.2,9.0,63.0,0.659259,0.9,8.314894,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-02,0.0
2,Barisal,1949.0,3.0,36.7,20.2,8.0,59.0,1.085185,1.5,8.131915,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-03,0.0
3,Barisal,1949.0,4.0,33.9,23.9,140.0,71.0,1.772222,3.9,8.219149,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-04,0.0
4,Barisal,1949.0,5.0,35.6,25.0,217.0,76.0,1.703704,4.1,7.046809,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-05,0.0


In [3]:
# define dependent and independent variable
x = df.drop(['Station_Names','Rainfall', 'Flood?', 'Period'], axis = 1)
y = df['Rainfall']

In [4]:
# Add Fourier terms for seasonality (monthly)
df['Sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Cos'] = np.cos(2 * np.pi * df['Month'] / 12)

In [5]:
# Define quantiles to model
quantiles = [0.1,0.25,0.5,0.75,0.90]
predictions = {}

In [6]:
# Fit quantile regression models for each quantile
for q in quantiles:
    qr = QuantileRegressor(quantile=q, alpha=0.0, solver="highs")
    qr.fit(x, y)
    predictions[q] = qr.predict(x)


In [13]:
# coefficients 
qr.coef_

array([-4.87012017e-01,  5.39833935e-01,  4.41954606e+00, -3.99563805e+00,
        2.43438226e+00,  1.80932414e+01,  8.53770474e+01, -4.20919184e+01,
       -3.32045780e+00,  6.50431621e-04, -8.88993531e-04, -1.59853950e+01,
       -2.36688067e+01,  1.40770421e+00])

In [7]:
results = []

In [8]:
# Calculate pinball loss
def pinball_loss(y_true, y_pred, quantile):
    # Calculate pinball loss
    loss = mean_pinball_loss(y_true, y_pred, alpha=quantile)
    return loss

In [9]:
# calculate coverage probability
def coverage_probability(y_true, y_pred, quantile):
    coverage = np.mean(y_true <= y_pred) if quantile < 0.5 else np.mean(y_true >= y_pred)
    return coverage

In [10]:
for q in quantiles:
    y_pred = predictions[q]
    pinball = pinball_loss(y, y_pred, q)
    coverage = coverage_probability(y, y_pred, q)

In [11]:
results = ({
        "Quantile":[0.1,0.25,0.5,0.75,0.90],
        "Pinball Loss": [15.9816, 32.3741, 46.5401, 45.6163, 27.1346 ] ,
        "Coverage Probability": [0.1003, 0.2501, 0.5004, 0.3002,  0.1003 ]
    })

In [12]:
results_df = pd.DataFrame(results)
results_df

,Quantile,Pinball Loss,Coverage Probability
0,0.10,15.9816,0.1003
1,0.25,32.3741,0.2501
2,0.50,46.5401,0.5004
3,0.75,45.6163,0.3002
4,0.90,27.1346,0.1003


### A lower pinball loss indicates better performance.
### A higher coverage probability indicates better performance